In [ ]:
import lyricsgenius as lg
import pandas as pd
import numpy as np
import re
import json
import time
import warnings
import requests
warnings.simplefilter(action='ignore', category=FutureWarning)
GENIUS_TOKEN = "Y80WiHQGGXOuJZ1CHjuO1U5IUGnW6I1ftwCxCFBTB88VZHYL8XxBewoBibWrh05O"
genius = lg.Genius(GENIUS_TOKEN)
lyricsDF = pd.DataFrame(columns=["Album", "Artist","Song #","Song Name", "Lyrics"])
reviewsDF = pd.read_csv('./fantano_reviews.csv', index_col=0)
reviewsDF.sort_values(by=['Artist', 'Album'], inplace=True)

In [ ]:
# Get lyrics for each song in each album
for index,value in reviewsDF["Album"].items():
    album = str(value).strip()
    artist = reviewsDF["Artist"][index].strip()
    song_count = 0
    try:
        time.sleep(6)
        songs = genius.search_album(album,artist)
        for track in songs.tracks:
            song_count += 1
            count = str(song_count) + "/" + str(len(songs.tracks))
            song_name = track.song.title
            lyrics = re.sub(r"\[.+\]\n", ' ', track.to_text()) # Remove [Verse 1], [Chorus], etc.
            if lyrics.endswith("You might also like"): # Remove "You might also like" at the end
                lyrics = lyrics[:-19]
            lyrics = lyrics.strip().replace("\n"," ")
            row = {"Album": album, "Artist": artist, "Song #": count, "Song Name": song_name, "Lyrics": lyrics}
            lyricsDF = lyricsDF.append(row, ignore_index=True)
    except Exception as e:
        print(e)
        pass

In [ ]:
# Add back into reviewsDF after it has been saved
lyrics = re.sub(r"\[[^\]]*\]", ' ', lyrics)
lyrics = re.sub(r"\\", '', lyrics) #Remove \
lyrics = re.sub(r"[0-9][0-9]Embed","",lyrics) # Remove embed cods
lyrics = re.sub(r"Translations.*Lyrics","",lyrics) # Remove translations

In [ ]:
lyricsDF.to_csv('./album_lyrics.csv')

In [ ]:
def get_vocab(corpus):
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    vocab = vectorizer.get_feature_names()
    
    # lets just get rid of any short words
    return [word for word in vocab if len(word) > 2 and not word.isdigit()]
    
def get_rhymes(word):
    
    url = f"https://api.datamuse.com/words?rel_rhy={word.lower()}"
    
    response = requests.get(url)
    
    return response.json()

In [ ]:
# Get the rhyming words for each song to add to lyricsDF
songs_list = lyricsDF["Lyrics"].tolist()
for index,value in reviewsDF["Album"].items():
    rhyme = 0
    lyrics = songs_list[index]
    vocab = get_vocab([lyrics])

    for word in vocab:
        rhymes_response = get_rhymes(word)
        if len(rhymes_response) > 0: # If there are rhymes for the word
            rhyming_words = [rhyme['word'] for rhyme in rhymes_response] + [word]
    